# Build System


### Set the build parameters

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_1_1'

### Connect to the Hardware

Using the parameters set above, set up and connect to the hardware. This is robust to allow for different ChipWhisperer boards and/or targets.

In [2]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")

(ChipWhisperer NAEUSB WARNING|File naeusb.py:713) Your firmware (0.62) is outdated - latest is 0.64 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍


In [3]:
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
else:
    prog = None

In [4]:
import time
time.sleep(0.05)
scope.default_setup()
scope.adc.samples = 30000
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

### Building the Firmware

In [6]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 SBOX2=0


Building for platform CWNANO with CRYPTO_TARGET=NONE
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Building for platform CWNANO with CRYPTO_TARGET=NONE
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
make[1]: '.dep' is up to date.
Building for platform CWNANO with CRYPTO_TARGET=NONE
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-aes.c ...
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32f0_nano/stm32f0_hal_nano.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32f0/stm32f0_hal_lowlevel.c ...
-e Done!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc

/usr/lib/gcc/arm-none-eabi/10.3.1/../../../arm-none-eabi/bin/ld: objdir-CWNANO/simpleserial-aes.o: in function `get_mask':
/home/beth-c132/S_Box_Analysis/chipwhisperer_minimal/firmware/simpleserial-aes/simpleserial-aes.c:27: undefined reference to `aes_indep_mask'
/usr/lib/gcc/arm-none-eabi/10.3.1/../../../arm-none-eabi/bin/ld: objdir-CWNANO/simpleserial-aes.o: in function `get_key':
/home/beth-c132/S_Box_Analysis/chipwhisperer_minimal/firmware/simpleserial-aes/simpleserial-aes.c:33: undefined reference to `aes_indep_key'
/usr/lib/gcc/arm-none-eabi/10.3.1/../../../arm-none-eabi/bin/ld: objdir-CWNANO/simpleserial-aes.o: in function `get_pt':
/home/beth-c132/S_Box_Analysis/chipwhisperer_minimal/firmware/simpleserial-aes/simpleserial-aes.c:39: undefined reference to `aes_indep_enc_pretrigger'
/usr/lib/gcc/arm-none-eabi/10.3.1/../../../arm-none-eabi/bin/ld: /home/beth-c132/S_Box_Analysis/chipwhisperer_minimal/firmware/simpleserial-aes/simpleserial-aes.c:47: undefined reference to `aes_inde

CalledProcessError: Command 'b'cd ../firmware/simpleserial-aes\nmake PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 SBOX2=0\n'' returned non-zero exit status 2.

### Upload Firmware

In [40]:
cw.program_target(scope, prog, "../firmware/simpleserial-base/simpleserial-base-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F03xx4/03xx6
Extended erase (0x44), this can take ten seconds or more
Attempting to program 7003 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 7003 bytes


### Define the Capture trace function

In [41]:
trace_array = []
for _ in range(100):
    msg = bytearray([0] + [0]*15)
    scope.arm()
    target.simpleserial_write('p', msg)
    scope.capture()
    # print(scope.get_last_trace())
    trace = scope.get_last_trace()
    target.simpleserial_write('x', bytearray([0]*16))
    trace_array.append(trace)
print("All done! :)")

All done! :)
